In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv
import re


options = Options()
service = Service()  
driver = webdriver.Chrome(service=service, options=options)

csv_file_path = 'products_data.csv'
fieldnames = ['UPC', 'Cost', 'URL', 'URL of Product', 'Title', 'Stock status']

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    try:
        driver.get('https://www.mxwholesale.co.uk/collections/all')

        while True:
            product_links = []
            product_elements = driver.find_elements(By.XPATH, '//a[contains(@href, "/products/")]')
            for index, element in enumerate(product_elements):
                if index % 2 == 0:  
                    product_links.append(element.get_attribute('href'))

            for href in product_links:
                print(f"Processing link: {href}")

                driver.get(href)

                html_content = driver.page_source
                soup = BeautifulSoup(html_content, 'html.parser')

                product_title = soup.find('h1', class_='product-single__title').get_text(strip=True) if soup.find('h1', class_='product-single__title') else 'Title not found'
                image_tag = soup.find('img', class_='lazyautosizes')
                image_url = image_tag['data-photoswipe-src'] if image_tag else 'Image URL not found'
                if image_url.startswith("//"):
                    image_url = image_url[2:]
                barcode_paragraph = soup.find('p', string=lambda x: x and 'Barcode:' in x)
                barcode = barcode_paragraph.get_text(strip=True).replace('Barcode: ', '') if barcode_paragraph else 'Barcode not found'
                td_tags = soup.find_all('td')
                last_unit_price = td_tags[-1].get_text(strip=True) if td_tags else 'Unit price not found'
                
                
                last_unit_price = re.sub(r'[^\d.,]', '', last_unit_price)  

                
                stock_message = 'Stock status not found'
                stock_message_element = soup.select_one('.product-savings .stock-message')
                if stock_message_element:
                    stock_message = stock_message_element.get_text(strip=True)

                # Print extracted data
                print(f"Product Title: {product_title}")
                print(f"Image URL: {image_url}")
                print(f"UPC: {barcode}")
                print(f"Last Unit Price: {last_unit_price}")
                print(f"Stock Status: {stock_message}")
                print("-----")

                # Write to CSV
                writer.writerow({
                    'UPC': barcode,
                    'Cost': last_unit_price,
                    'URL': href,
                    'URL of Product': image_url,
                    'Title': product_title,
                    'Stock status': stock_message
                })

                driver.back()
                time.sleep(2)

            try:
                load_more_button = driver.find_element(By.CLASS_NAME, 'btn-load-more')
                load_more_button.click()
                print("Clicked 'Load more products' button.")
                time.sleep(2)
            except Exception as e:
                print("No more 'Load more products' button found or all products have been loaded.")
                break

    finally:
        driver.quit()
